In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('merged_data.csv')

In [3]:
df.shape

(121838, 32)

In [4]:
df['seq'] = df['sequence'].str[1:6]

In [5]:
df.head()

,transcript_id,transcript_position,sequence,avg_1-flank_length,avg_1-flank_std,avg_1-flank_mean,avg_central_length,avg_central_std,avg_central_mean,avg_1+flank_length,...,std_1-flank_mean,std_central_length,std_central_std,std_central_mean,std_1+flank_length,std_1+flank_std,std_1+flank_mean,gene_id,label,seq
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,4.944365,0.005364,3.302671,2.765244,0.004566,2.906225,2.522168,ENSG00000004059,0,AGACC
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,2.981182,0.003767,1.237045,3.526528,0.004528,1.466295,2.492530,ENSG00000004059,0,AAACT
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,1.492577,0.005035,0.893481,3.203816,0.004497,0.800496,1.484341,ENSG00000004059,0,AAACA
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006102,...,3.202963,0.004612,1.393868,1.928009,0.003595,0.741771,3.494937,ENSG00000004059,0,GAACA
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,2.664643,0.005407,1.753894,2.177236,0.005908,1.341822,2.379045,ENSG00000004059,0,GGACA


In [6]:
df_le = df.copy()
holder = df_le["seq"].values.reshape(-1, 1)

label_encoder = LabelEncoder()

temp = pd.DataFrame(label_encoder.fit_transform(holder))

temp.columns = ["seq_le"]

df_le = df_le.join(temp)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
df_le = df_le.drop(["sequence", "seq"], axis=1)

In [8]:
df_le.columns

Index(['transcript_id', 'transcript_position', 'avg_1-flank_length',
       'avg_1-flank_std', 'avg_1-flank_mean', 'avg_central_length',
       'avg_central_std', 'avg_central_mean', 'avg_1+flank_length',
       'avg_1+flank_std', 'avg_1+flank_mean', 'med_1-flank_length',
       'med_1-flank_std', 'med_1-flank_mean', 'med_central_length',
       'med_central_std', 'med_central_mean', 'med_1+flank_length',
       'med_1+flank_std', 'med_1+flank_mean', 'std_1-flank_length',
       'std_1-flank_std', 'std_1-flank_mean', 'std_central_length',
       'std_central_std', 'std_central_mean', 'std_1+flank_length',
       'std_1+flank_std', 'std_1+flank_mean', 'gene_id', 'label', 'seq_le'],
      dtype='object')

In [9]:
train, test = train_test_split(df_le, test_size=0.2)

In [10]:
train.shape

(97470, 32)

In [11]:
test.shape

(24368, 32)

In [12]:
len(train[train['label'] == 0])

93145

In [13]:
len(train[train['label'] == 1])

4325

In [14]:
1 - 4332/93138

0.9534883720930233

In [17]:
X_train = train.drop(['transcript_id', 'gene_id', 'label'], axis=1)
y_train = train['label']

X_test = test.drop(['transcript_id', 'gene_id', 'label'], axis=1)
y_test = test['label']

In [18]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampledtrain, y_resampledtrain = smote.fit_resample(X_train, y_train)
X_resampledtest, y_resampledtest = smote.fit_resample(X_test, y_test)


In [60]:
original_class_counts = y_train.value_counts()
resampled_class_counts = pd.Series(y_resampledtrain).value_counts()

print("Original Class Distribution:")
print(original_class_counts)

print("\nResampled Class Distribution:")
print(resampled_class_counts)

Original Class Distribution:
label
0    93082
1     4388
Name: count, dtype: int64

Resampled Class Distribution:
label
0    93082
1    93082
Name: count, dtype: int64


In [19]:
original_class_counts = y_test.value_counts()
resampled_class_counts = pd.Series(y_resampledtest).value_counts()

print("Original Class Distribution:")
print(original_class_counts)

print("\nResampled Class Distribution:")
print(resampled_class_counts)

Original Class Distribution:
label
0    23218
1     1150
Name: count, dtype: int64

Resampled Class Distribution:
label
0    23218
1    23218
Name: count, dtype: int64


In [22]:
transcript_id_train = train['transcript_id']
gene_id_train = train['gene_id']

In [31]:
resampled_train = pd.DataFrame(data=X_resampledtrain, columns=X_train.columns)
resampled_train['label'] = y_resampledtrain

transcript_id_train.reset_index(drop=True, inplace=True)
gene_id_train.reset_index(drop=True, inplace=True)

resampled_train['transcript_id'] = transcript_id_train
resampled_train['gene_id'] = gene_id_train
resampled_train.to_csv('resampled_traindata.csv', index=False)


In [32]:
transcript_id_test = test['transcript_id']
gene_id_test = test['gene_id']

In [33]:
resampled_test = pd.DataFrame(data=X_resampledtest, columns=X_test.columns)
resampled_test['label'] = y_resampledtest

transcript_id_test.reset_index(drop=True, inplace=True)
gene_id_test.reset_index(drop=True, inplace=True)

resampled_test['transcript_id'] = transcript_id_test
resampled_test['gene_id'] = gene_id_test
resampled_test.to_csv('resampled_testdata.csv', index=False)

In [34]:
resampled_test

,transcript_position,avg_1-flank_length,avg_1-flank_std,avg_1-flank_mean,avg_central_length,avg_central_std,avg_central_mean,avg_1+flank_length,avg_1+flank_std,avg_1+flank_mean,...,std_central_length,std_central_std,std_central_mean,std_1+flank_length,std_1+flank_std,std_1+flank_mean,seq_le,label,transcript_id,gene_id
0,365,0.008521,3.519315,105.167123,0.006436,3.179589,98.180822,0.007252,2.282192,88.631507,...,0.003475,0.870872,2.559840,0.004346,0.401907,2.231093,0,0,ENST00000330263,ENSG00000053372
1,1736,0.009964,3.383571,107.750000,0.006947,3.978214,102.664286,0.005644,2.473929,86.046429,...,0.003299,0.969311,3.431911,0.002409,0.672120,2.397243,1,0,ENST00000267812,ENSG00000140259
2,1746,0.008331,4.309636,124.345455,0.010254,3.771636,131.181818,0.008321,3.463273,91.789091,...,0.007206,1.405330,2.848227,0.004373,1.549521,1.972140,5,0,ENST00000326005,ENSG00000180304
3,243,0.009277,7.715652,114.847826,0.009479,3.457826,95.847826,0.007170,2.321522,86.206522,...,0.004887,0.750234,2.605491,0.004376,0.800272,2.673981,7,0,ENST00000262265,ENSG00000104872
4,1140,0.007623,7.957727,121.159091,0.006555,3.804091,99.190909,0.007035,1.817273,92.143182,...,0.003999,2.189332,3.058358,0.003624,0.552150,1.782796,8,0,ENST00000358007,ENSG00000213782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46431,547,0.007428,3.972655,116.275210,0.007836,5.352416,120.608219,0.006589,3.177833,88.688501,...,0.004538,2.082458,4.066502,0.003730,1.765706,3.009236,11,1,NaN,NaN
46432,2118,0.008322,4.966005,117.987975,0.008317,7.339242,121.602817,0.007070,3.441620,81.662968,...,0.004151,3.401207,4.603903,0.003850,1.654510,3.688426,5,1,NaN,NaN
46433,759,0.006232,2.303664,92.629415,0.006353,3.821207,104.140763,0.006249,2.166976,89.907651,...,0.003517,1.156852,2.603967,0.003518,0.821677,2.645505,14,1,NaN,NaN
46434,1640,0.005725,3.164280,109.876653,0.006673,7.749766,118.590498,0.006791,3.605968,88.479078,...,0.003078,2.296158,2.785564,0.003579,1.987396,2.227137,16,1,NaN,NaN
